In [2]:
import cv2


# Face Detection


In [3]:
# Method to draw boundary around the detected feature
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    coords = []
    for (x, y, w, h) in features:
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
        cv2.putText(img, text, (x, y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        coords = [x, y, w, h]
    return coords

In [4]:
# Method to detect the features
def detect(img, faceCascade, eyeCascade, noseCascade, mouthCascade):
    color = {"blue":(255,0,0), "red":(0,0,255), "green":(0,255,0), "white":(255,255,255)}
    coords = draw_boundary(img, faceCascade, 1.1, 10, color['blue'], "Face")

    if len(coords)==4:
        roi_img = img[coords[1]:coords[1]+coords[3], coords[0]:coords[0]+coords[2]]
        coords = draw_boundary(roi_img, eyeCascade, 1.1, 12, color['red'], "Eye")
        coords = draw_boundary(roi_img, noseCascade, 1.1, 4, color['green'], "Nose")
        coords = draw_boundary(roi_img, mouthCascade, 1.1, 20, color['white'], "Mouth")
    return img

In [5]:
# Loading classifiers
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eyesCascade = cv2.CascadeClassifier('haarcascade_eye.xml')
noseCascade = cv2.CascadeClassifier('Nariz.xml')
mouthCascade = cv2.CascadeClassifier('Mouth.xml')

In [6]:
# Capturing real time video stream
video_capture = cv2.VideoCapture(0)

while True:
    # Reading image from video stream
    _, img = video_capture.read()
    # Call method we defined above
    img = detect(img, faceCascade, eyesCascade, noseCascade, mouthCascade)
    # Writing processed image in a new window
    cv2.imshow("face detection", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
video_capture.release()
cv2.destroyAllWindows()

# Dataset


In [7]:
# Method to generate dataset to recognize a person
def generate_dataset(img, id, img_id):
    cv2.imwrite("C:\\Users\\DELL\\Desktop\\febmain\\dataset\\user."+str(id)+"."+str(img_id)+".jpg", img)
    
# Method to draw boundary around the detected feature
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    coords = []

    for (x, y, w, h) in features:
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
        cv2.putText(img, text, (x, y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        coords = [x, y, w, h]
    return coords


In [8]:
# Method to detect the features
def detect(img, faceCascade, img_id):
    color = {"blue":(255,0,0), "red":(0,0,255), "green":(0,255,0), "white":(255,255,255)}
    coords = draw_boundary(img, faceCascade, 1.1, 10, color["blue"], "Face")
    
    if len(coords)==4:
        roi_img = img[coords[1]:coords[1]+coords[3], coords[0]:coords[0]+coords[2]]
        user_id = 1
        generate_dataset(roi_img, user_id, img_id)

    return img


In [10]:
# Loading classifiers
faceCascade = cv2.CascadeClassifier('D:\\Minor Project 6th sem Attendance\\febmain\\recognition-main\\haarcascade_frontalface_default.xml')



In [12]:
# Capturing real time video stream.
video_capture = cv2.VideoCapture(0)

img_id = 0

while True:
    if img_id % 50 == 0:
        print("Collected ", img_id," images")
    _, img = video_capture.read()
    img = detect(img, faceCascade, img_id)
    cv2.imshow("face detection", img)
    img_id += 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

Collected  0  images
Collected  50  images
Collected  100  images
Collected  150  images


# Classifier


In [13]:
import numpy as np
from PIL import  Image
import os, cv2


In [14]:
# Method to train custom classifier to recognize face
def train_classifer(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
        
        imageNp = cv2.resize(imageNp, (200, 200))

        faces.append(imageNp)
        ids.append(id)

    ids = np.array(ids)

    # Train and save classifier
    clf = cv2.face.EigenFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("C:\\Users\\DELL\\Desktop\\febmain\\classifier1.xml")


train_classifer("C:\\Users\\DELL\\Desktop\\febmain\\dataset")

# Recognize


In [22]:
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #gray_img = cv2.resize(gray_img, (200,200))
    
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbors)
    coords = []
    
    for (x, y, w, h) in features:
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
        id, _ = clf.predict(cv2.resize((gray_img[y:y+h, x:x+w]), (200,200)))

        if id==1:
            cv2.putText(img, "Suyan", (x, y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        elif id==2:
            cv2.putText(img, "ronaldo", (x, y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "Face Not Recognized", (x, y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)

        coords = [x, y, w, h]

    return coords

def recognize(img, clf, faceCascade):
    color = {"blue": (255, 0, 0), "red": (0, 0, 255), "green": (0, 255, 0), "white": (255, 255, 255)}
    coords = draw_boundary(img, faceCascade, 1.1, 10, color["white"], "Face", clf)
    print(coords)
    return img


In [23]:
# Loading classifier
faceCascade = cv2.CascadeClassifier('D:\\Minor Project 6th sem Attendance\\febmain\\recognition-main\\haarcascade_frontalface_default.xml')

clf = cv2.face.EigenFaceRecognizer_create()
clf.read("C:\\Users\\DELL\\Desktop\\febmain\\classifier1.xml")

video_capture = cv2.VideoCapture(0)

while True:
    _, img = video_capture.read()
    img = recognize(img, clf, faceCascade)
    cv2.imshow("face detection", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

[192, 104, 198, 198]
[191, 104, 199, 199]
[190, 103, 201, 201]
[189, 105, 198, 198]
[192, 105, 197, 197]
[190, 105, 199, 199]
[191, 106, 195, 195]
[192, 105, 196, 196]
[192, 106, 197, 197]
[190, 106, 195, 195]
[192, 106, 196, 196]
[194, 106, 193, 193]
[192, 105, 199, 199]
[192, 107, 192, 192]
[189, 107, 194, 194]
[191, 107, 193, 193]
[191, 107, 195, 195]
[189, 106, 196, 196]
[190, 107, 196, 196]
[190, 108, 194, 194]
[191, 110, 192, 192]
[190, 109, 194, 194]
[193, 110, 192, 192]
[189, 108, 196, 196]
[188, 108, 197, 197]
[191, 110, 192, 192]
[187, 108, 196, 196]
[186, 109, 195, 195]
[189, 108, 195, 195]
[186, 107, 194, 194]
[188, 109, 190, 190]
[181, 108, 195, 195]
[184, 110, 191, 191]
[183, 109, 196, 196]
[182, 108, 197, 197]
[181, 109, 195, 195]
[180, 107, 199, 199]
[182, 108, 196, 196]
[180, 108, 198, 198]
[182, 108, 197, 197]
[181, 107, 199, 199]
[180, 107, 202, 202]
[182, 108, 196, 196]
[181, 107, 197, 197]
[181, 108, 197, 197]
[180, 106, 200, 200]
[182, 110, 194, 194]
[180, 107, 19

[]
[]
[]
[165, 175, 118, 118]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[162, 183, 115, 115]
[]
[159, 184, 116, 116]
[]
[]
[]
[]
[]
[]
[154, 188, 112, 112]
[]
[155, 192, 112, 112]
[153, 191, 116, 116]
[153, 192, 115, 115]
[156, 193, 115, 115]
[]
[157, 192, 114, 114]
[158, 191, 115, 115]
[]
[156, 193, 115, 115]
[156, 192, 114, 114]
[]
[]
[159, 193, 113, 113]
[159, 195, 114, 114]
[156, 195, 117, 117]
[159, 196, 116, 116]
[]
[159, 196, 114, 114]
[156, 197, 116, 116]
[156, 197, 116, 116]
[157, 197, 115, 115]
[]
[]
[160, 197, 114, 114]
[162, 201, 111, 111]
[159, 203, 114, 114]
[156, 210, 110, 110]
[151, 216, 111, 111]
[155, 232, 94, 94]
[150, 243, 96, 96]
[]
[194, 94, 203, 203]
[195, 97, 192, 192]
[199, 96, 188, 188]
[199, 95, 189, 189]
[]
[196, 91, 196, 196]
[199, 93, 191, 191]
[199, 95, 191, 191]
[266, 316, 54, 54]
[198, 95, 195, 195]
[199, 94, 196, 196]
[199, 94, 198, 198]
[203, 94, 195, 195]
[203, 95, 196, 196]
[202, 95, 195, 195]
[202, 95, 197, 197]
[199, 95,